```
This file is part of sscx-connectome-manipulations.

SPDX-License-Identifier: Apache-2.0
Copyright (c) 2024 Blue Brain Project/EPFL
```

# Analysis notebook of current injection simulations

ℹ️ Reproduction of interneuron rewiring results described in [Pokorny et al. (2024)](https://doi.org/10.1101/2024.05.24.593860)

<u>Requirements</u>:
- [BlueETL](https://github.com/BlueBrain/blueetl) (Python venv)
- [Blue Brain SNAP](https://github.com/BlueBrain/snap) (Python venv)
- [CortexETL](https://github.com/BlueBrain/cortexetl) (Cloned repo)
- Simulation results (see instructions in `README.md` on how to rerun the simulations; also available on [Zenodo](https://zenodo.org/))

In [1]:
from bluepysnap import Simulation
import numpy as np
import matplotlib.pyplot as plt
import os

ℹ️ Set path to cloned CortexETL repository below

In [2]:
# Set path to cloned CortexETL repository
import sys
sys.path.append('/gpfs/bbp.cscs.ch/home/pokorny/JupyterLab/git/cortex_etl')
import cortex_etl as c_etl

Run data analysis using CortexETL

ℹ️ Before running the analyses, set paths in `../configs/cortexetl_analysis_config__<...>.yaml` accordingly:
- "simulation_campaign" ... pointing to `config.json` from respective simulation campaign
  - In `config.json`: Make sure that "path_prefix" is pointing to actual simulation campaign folder!
- "output" ... pointing to some output folder to which the analysis results will be written

In [4]:
# Run (or reload) analysis
ma_orig = c_etl.run_from_file(f"../configs/cortexetl_analysis_config__orig_{sim_spec}.yaml", loglevel="WARNING")
ma_manip = c_etl.run_from_file(f"../configs/cortexetl_analysis_config__manip_{sim_spec}.yaml", loglevel="WARNING")

Once completed successfully, the results will be stored in the following folder and can be used to reproduce results figures using `connectome_manipulator_figs_functional.ipynb`

In [37]:
print("Available analysis results:")
for _name, _anlyz in ma_orig.analyzers.items():
    print(f'  Orig "{_name}": {_anlyz.analysis_config.output}')
for _name, _anlyz in ma_manip.analyzers.items():
    print(f'  Manip "{_name}": {_anlyz.analysis_config.output}')

Available analysis results:
  Orig "spikes": /gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/ResultsTMP/interneuron_rewiring/analysis/orig/Ca1p05_Pfr0p3_Rou0p4/spikes
  Manip "spikes": /gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/ResultsTMP/interneuron_rewiring/analysis/manip/Ca1p05_Pfr0p3_Rou0p4/spikes
